#Power Plant ML Pipeline Application
This is an end-to-end example of using a number of different machine learning algorithms to solve a supervised regression problem.

###Table of Contents

- *Step 1: Business Understanding*
- *Step 2: Load Your Data*
- *Step 3: Explore Your Data*
- *Step 4: Visualize Your Data*
- *Step 5: Data Preparation*
- *Step 6: Data Modeling*
- *Step 7: Tuning and Evaluation*
- *Step 8: Deployment*



*We are trying to predict power output given a set of readings from various sensors in a gas-fired power generation plant.  Power generation is a complex process, and understanding and predicting power output is an important element in managing a plant and its connection to the power grid.*

More information about Peaker or Peaking Power Plants can be found on Wikipedia https://en.wikipedia.org/wiki/Peaking_power_plant


Given this business problem, we need to translate it to a Machine Learning task.  The ML task is regression since the label (or target) we are trying to predict is numeric.


The example data is provided by UCI at [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)

You can read the background on the UCI page, but in summary we have collected a number of readings from sensors at a Gas Fired Power Plant

(also called a Peaker Plant) and now we want to use those sensor readings to predict how much power the plant will generate.


More information about Machine Learning with Spark can be found in the programming guide in the [SparkML Guide](https://spark.apache.org/docs/latest/mllib-guide.html)


*Please note this example only works with Spark version 1.4 or higher*

In [2]:
assert int(sc.version.replace(".", "")) >= 140, "Spark 1.4.0+ is required to run this notebook. Please attach it to a Spark 1.4.0+ cluster."

##Step 1: Business Understanding
The first step in any machine learning task is to understand the business need. 

As described in the overview we are trying to predict power output given a set of readings from various sensors in a gas-fired power generation plant.

The problem is a regression problem since the label (or target) we are trying to predict is numeric

##Step 2: Load Your Data
Now that we understand what we are trying to do, we need to load our data and describe it, explore it and verify it.

Since the dataset is relatively small, we will use the upload feature in Databricks to upload the data as a table.

First download the Data Folder from [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)

The file is a multi-tab Excel document so you will need to save each tab as a Text file export. 

I prefer exporting as a Tab-Separated-Values (TSV) since it is more consistent than CSV.

Call each file Folds5x2_pp<Sheet 1..5>.tsv and save to your machine.

Go to the Databricks Menu > Tables > Create Table

Select Datasource as "File"

Upload *ALL* 5 files at once.

See screenshots below:


**2.1. Create Table**
  _________________

When you import your data, name your table `power_plant`, specify all of the columns with the datatype `Double` and make sure you check the `First row is header` box.

![alt text](http://training.databricks.com/databricks_guide/1_4_ML_Power_Plant_Import_Table.png)

**2.2. Review Schema**
  __________________

Your table schema and preview should look like this after you click ```Create Table```:

![alt text](http://training.databricks.com/databricks_guide/1_4_ML_Power_Plant_Import_Table_Schema.png)

Now that your data is loaded let's explore it.

#### Step 2: (Load your Data Alternative Option):

If you did Step 2 already you can skip down to Step 3. If you want to skip the data import and just load the data from our public datasets in S3 use the cell below.

In [9]:
from collections import namedtuple
sqlContext.sql("DROP TABLE IF EXISTS power_plant")
dbutils.fs.rm("dbfs:/user/hive/warehouse/power_plant", True)
PowerPlantTable=namedtuple("PowerPlantTable", ["AT", "V", "AP", "RH", "PE"])
rawTextRdd = sc.textFile("dbfs:/databricks-datasets/power-plant/data/")
powerPlant=rawTextRdd\
  .map(lambda x: x.split("\t"))\
  .filter(lambda line: line[0] != "AT")\
  .map(lambda line: PowerPlantTable(float(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])))\
  .toDF()
#powerPlant.write.saveAsTable("power_plant")
sqlContext.registerDataFrameAsTable(powerPlant, "power_plant")


##Step 3: Explore Your Data
Now that we understand what we are trying to do, we need to load our data and describe it, explore it and verify it.

In [11]:
%sql 
--We can use %sql to query the rows

SELECT * FROM power_plant

We can use the SQL desc command to describe the schema

In [13]:
%sql desc power_plant

**Schema Definition**

Our schema definition from UCI appears below:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

PE is our label or target. This is the value we are trying to predict given the measurements.

*Reference [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)*

Let's do some basic statistical analysis of all the columns. 

We can use the describe function with no parameters to get some basic stats for each column like count, mean, max, min and standard deviation. The describe function is a method attached to a dataframe. More information can be found in the [Spark API docs](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrame)

In [16]:
display(sqlContext.table("power_plant").describe())

##Step 4: Visualize Your Data

To understand our data, we will look for correlations between features and the label.  This can be important when choosing a model.  E.g., if features and a label are linearly correlated, a linear model like Linear Regression can do well; if the relationship is very non-linear, more complex models such as Decision Trees can be better. We use Databrick's built in visualization to view each of our predictors in relation to the label column as a scatter plot to see the correlation between the predictors and the label.

In [18]:
%sql select AT as Temperature, PE as Power from power_plant

It looks like there is strong linear correlation between temperature and Power Output

In [20]:
%sql select V as ExhaustVacuum, PE as Power from power_plant;

The linear correlation is not as strong between Exhaust Vacuum Speed and Power Output but there is some semblance of a pattern.

In [22]:
%sql select AP Pressure, PE as Power from power_plant;

In [23]:
%sql select RH Humidity, PE Power from power_plant;

...and atmospheric pressure and relative humidity seem to have little to no linear correlation

##Step 5: Data Preparation

The next step is to prepare the data. Since all of this data is numeric and consistent this is a simple task for us today.

We will need to convert the predictor features from columns to Feature Vectors using the org.apache.spark.ml.feature.VectorAssembler

The VectorAssembler will be the first step in building our ML pipeline.

In [26]:
from pyspark.ml.feature import VectorAssembler

dataset = sqlContext.table("power_plant")

vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")


##Step 6: Data Modeling
Now let's model our data to predict what the power output will be given a set of sensor readings

Our first model will be based on simple linear regression since we saw some linear patterns in our data based on the scatter plots during the exploration stage.

In [28]:
#First let's hold out 20% of our data for testing and leave 80% for training

(split20, split80) = dataset.randomSplit((0.20, 0.80), seed=1800009193L)



In [29]:
# Let's cache these datasets for performance
testSet = split20.cache()
trainingSet = split80.cache()

In [30]:
# ***** LINEAR REGRESSION MODEL ****

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline

# Let's initialize our linear regression learner
lr = LinearRegression()



In [31]:
# We use explain params to dump the parameters we can use
print(lr.explainParams())

The cell below is based on the Spark ML pipeline API. More information can be found in the Spark ML Programming Guide at https://spark.apache.org/docs/latest/ml-guide.html

In [33]:
# Now we set the parameters for the method
lr.setPredictionCol("Predicted_PE")\
  .setLabelCol("PE")\
  .setMaxIter(100)\
  .setRegParam(0.1)


# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()

lrPipeline.setStages([vectorizer, lr])


# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSet)



Since Linear Regression is Simply a Line of best fit over the data that minimizes the square of the error, given multiple input dimensions we can express each predictor as a line function of the form:

\\[ y = a + b x_1 + b x_2 + b x_i ... \\]

where a is the intercept and b are coefficients.

To express the coefficients of that line we can retrieve the Estimator stage from the PipelineModel and express the weights and the intercept for the function.

In [35]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept
print(intercept)

In [36]:
# The coefficents (i.e. weights) are as follows:

weights = lrModel.stages[1].weights.toArray()


featuresNoLabel = [col for col in dataset.columns if col != "PE"]

coefficents = sc.parallelize(weights).zip(sc.parallelize(featuresNoLabel))

# Now let's sort the coeffecients from the most to the least

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents.sortByKey().collect():
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

Based on examining the output it shows there is a strong negative correlation between Atmospheric Temperature (AT) and Power Output.

But our other dimenensions seem to have little to no correlation with Power Output. Do you remember **Step 2: Explore Your Data**? When we visualized each predictor against Power Output using a Scatter Plot, only the temperature variable seemed to have a linear correlation with Power Output so our final equation seems logical.


Now let's see what our predictions look like given this model.

In [38]:
predictionsAndLabels = lrModel.transform(testSet)

display(predictionsAndLabels.select("AT", "V", "AP", "RH", "PE", "Predicted_PE"))

Now that we have real predictions we can use an evaluation metric such as Root Mean Squared Error to validate our regression model. The lower the Root Mean Squared Error, the better our model.

In [40]:
# Now let's compute some evaluation metrics against our test dataset

from pyspark.mllib.evaluation import RegressionMetrics

metrics = RegressionMetrics(predictionsAndLabels.select("Predicted_PE", "PE").rdd.map(lambda r: (float(r[0]), float(r[1]))))

rmse = metrics.rootMeanSquaredError
explainedVariance = metrics.explainedVariance
r2 = metrics.r2

print("Root Mean Squared Error: {}".format(rmse))
print("Explained Variance: {}".format(explainedVariance))
print("R2: {}".format(r2))

Generally a good model will have 68% of predictions within 1 RMSE and 95% within 2 RMSE of the actual value. Let's calculate and see if a RMSE of 4.51 meets this criteria.

In [42]:
# First we calculate the residual error and divide it by the RMSE
predictionsAndLabels.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmse)).registerTempTable("Power_Plant_RMSE_Evaluation")

In [43]:
%sql SELECT * from Power_Plant_RMSE_Evaluation

In [44]:
%sql -- Now we can display the RMSE as a Histogram. Clearly this shows that the RMSE is centered around 0 with the vast majority of the error within 2 RMSEs.
SELECT Within_RSME  from Power_Plant_RMSE_Evaluation

We can see this definitively if we count the number of predictions within + or - 1.0 and + or - 2.0 and display this as a pie chart:

In [46]:
%sql SELECT case when Within_RSME <= 1.0 and Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 and Within_RSME >= -2.0 then 2 else 3 end RSME_Multiple, COUNT(*) count  from Power_Plant_RMSE_Evaluation
group by case when Within_RSME <= 1.0 and Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 and Within_RSME >= -2.0 then 2 else 3 end

So we have 68% of our training data within 1 RMSE and 97% (68% + 29%) within 2 RMSE. So the model is pretty decent. Let's see if we can tune the model to improve it further.

#Step 7: Tuning and Evaluation

Now that we have a model with all of the data let's try to make a better model by tuning over several parameters.  We'll use the `ParamGridBuilder` and `CrossValidator` from [pyspark.ml.tuning](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.tuning) to perform grid search.

**ToDo:** Create a `CrossValidator` for `crossval`

**ToDo:**  Create a `ParamGridBuilder` for  `paramGrid`

**ToDo:**  Set `crossval`'s `estimatorParamMaps` to the `paramGrid`

In [49]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Let's set up our evaluator class to judge the model based on the best root mean squared error
regEval = RegressionEvaluator(predictionCol="Predicted_PE", labelCol="PE", metricName="rmse")

# Let's create our crossvalidator with 3 fold cross validation
crossval = # TODO
# Let's tune over our regularization parameter from 0.01 to 0.10
regParam = [x / 100.0 for x in range(1, 11)]

paramGrid = ()  #TODO
crossval.setEstimatorParamMaps()  #TODO

# Now let's create the model
cvModel = crossval.fit(trainingSet)

Now that we have tuned let's see what we got for tuning parameters and what our RMSE was versus our intial model

**ToDo:** Run this code

In [51]:
predictionsAndLabels = cvModel.transform(testSet)
metrics = RegressionMetrics(predictionsAndLabels.select("Predicted_PE", "PE").rdd)
                            
rmse = metrics.rootMeanSquaredError
explainedVariance = metrics.explainedVariance
r2 = metrics.r2

print "Root Mean Squared Error: {0}".format(rmse)
print "Explained Variance: {0}".format(explainedVariance)  
print "R2: {0}".format(r2)

So our initial untuned and tuned linear regression models are statistically identical.

Given that the only linearly correlated variable is Temperature, it makes sense try another machine learning method such a Decision Tree to handle non-linear data and see if we can improve our model.  We'll use [DecisionTreeRegressor](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor) for this purpose.

A Decision Tree creates a model based on splitting variables using a tree structure. We will first start with a single decision tree model.

Reference Decision Trees: https://en.wikipedia.org/wiki/Decision_tree_learning

**ToDo:** Create a `DecisionTreeRegressor` and set the stages for the pipeline.  Note that you can reuse `vectorizer` in the pipeline.  Feel free to change some of the parameters to compare performance.

In [53]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = # TODO
dt.setLabelCol("PE")
dt.setPredictionCol("Predicted_PE")
dt.setFeaturesCol("features")
dt.setMaxBins(100)

dtPipeline = Pipeline()
dtPipeline.setStages() #TODO

# Let's just reuse our CrossValidator
crossval.setEstimator(dtPipeline)

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [2, 3])
             .build())
crossval.setEstimatorParamMaps(paramGrid)

dtModel = crossval.fit(trainingSet)

Now let's see how our DecisionTree model compares to our LinearRegression model

**ToDo:** Run this code

In [55]:
predictionsAndLabels = dtModel.bestModel.transform(testSet)
metrics = RegressionMetrics(predictionsAndLabels.select("Predicted_PE", "PE").rdd)

rmse = metrics.rootMeanSquaredError
explainedVariance = metrics.explainedVariance
r2 = metrics.r2

print "Root Mean Squared Error: {0}".format(rmse)
print "Explained Variance: {0}".format(explainedVariance)  
print "R2: {0}".format(r2)




The line below will pull the Decision Tree model from the Pipeline as display it as an if-then-else string

**ToDo:** Run this code to see the decision tree

In [57]:
print dtModel.bestModel.stages[-1]._java_obj.toDebugString()

So our DecisionTree was slightly worse than our LinearRegression model (LR: 4.51 vs DT: 5.03). Maybe we can try an ensemble method such as Random Forest to see if we can strengthen our model by using an ensemble of weaker trees with weighting to reduce the error in our model.

*This will take about one minute to run.*

**ToDo:** Run this code.

In [59]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor()
rf.setLabelCol("PE")
rf.setPredictionCol("Predicted_PE")
rf.setFeaturesCol("features")
rf.setSeed(100088121L)
rf.setMaxDepth(8)
rf.setNumTrees(30)

rfPipeline = Pipeline()
rfPipeline.setStages([vectorizer, rf])

# Let's just resuse our CrossValidator
crossval.setEstimator(rfPipeline)

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxBins, [50, 100]) 
             .build())
crossval.setEstimatorParamMaps(paramGrid)

rfModel = crossval.fit(trainingSet)

In [60]:
predictionsAndLabels = rfModel.transform(testSet)
metrics = RegressionMetrics(predictionsAndLabels.select("Predicted_PE", "PE").rdd)

rmse = metrics.rootMeanSquaredError
explainedVariance = metrics.explainedVariance
r2 = metrics.r2

print "Root Mean Squared Error: {0}".format(rmse)
print "Explained Variance: {0}".format(explainedVariance)  
print "R2: {0}".format(r2)

In [61]:
print rfModel.bestModel.stages[-1]._java_obj.parent().explainParams()

In [62]:
print rfModel.bestModel.stages[-1]._java_obj.parent().getMaxBins()

In [63]:
# Model is quite large so this takes a couple minutes
# print rfModel.bestModel.stages[-1]._java_obj.toDebugString()

### Conclusion

Wow! So our best model is in fact our Random Forest tree model which uses an ensemble of 30 Trees with a depth of 8 to construct a better model than the single decision tree.

Datasource References:
* Pinar Tüfekci, Prediction of full load electrical power output of a base load operated combined cycle power plant using machine learning methods, International Journal of Electrical Power & Energy Systems, Volume 60, September 2014, Pages 126-140, ISSN 0142-0615, [Web Link](http://www.journals.elsevier.com/international-journal-of-electrical-power-and-energy-systems/)
* Heysem Kaya, Pinar Tüfekci , Sadik Fikret Gürgen: Local and Global Learning Methods for Predicting Power of a Combined Gas & Steam Turbine, Proceedings of the International Conference on Emerging Trends in Computer and Electronics Engineering ICETCEE 2012, pp. 13-18 (Mar. 2012, Dubai) [Web Link](http://www.cmpe.boun.edu.tr/~kaya/kaya2012gasturbine.pdf)